In [42]:
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix, classification_report

!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hksca\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hksca\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [43]:
df = pd.read_csv('Downloads/Train_Data.csv', keep_default_na=False)
df.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers,average_review_rating
0,7f890f73298badd692c39c3801ef1d28,Batman - Role Play - Cowl and Batarang,Mattel,£14.99,,4,1.0,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Rubies-Costume-Co-Flas...,"When kids aspire to be like Batman, it's easy ...",Technical Details Item Weight113 g Product Dim...,"When kids aspire to be like Batman, it's easy ...",http://www.amazon.co.uk/Rubies-Costume-Co-Flas...,,Item was broken // 3.0 // 2 April 2014 // By\n...,"{""seller""=>{""Seller_name_1""=>""a1 Toys"", ""Selle...",4.5
1,83e55a2edb6c13dfc1c46d1577321b2e,Doctor Who 12-inch 50th Anniversary Collector ...,Dr Who,,3 new,73,,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Doctor-Who-Classic-Dal...,Product Description This ultimate Doctor Who c...,Technical Details Item Weight1.4 Kg Product Di...,Product Description This ultimate Doctor Who c...,http://www.amazon.co.uk/Dr-Who-Dalek-Projectio...,,A wonderful collectors piece // 5.0 // 31 Aug....,,4.6
2,4bfaaf6558582a0f337ad59fcd51fcf7,Melissa & Doug Sticker Collection - Pink,Melissa & Doug,£4.76,15 new,47,1.0,Arts & Crafts > Paper & Stickers,http://www.amazon.co.uk/Melissa-Doug-Sticker-C...,"Product Description Cute couture, adorable ani...",Technical Details Item Weight299 g Product Dim...,"Product Description Cute couture, adorable ani...",http://www.amazon.co.uk/Melissa-Doug-Habitats-...,,Great sticker collection // 5.0 // 19 Jun. 201...,,4.7
3,4e6702cb7fe9278f16dedc39a8808c72,"Rock and Roll Cards, 30 Team",ClubKing Ltd,£4.49,2 new,2,1.0,Hobbies > Trading Cards & Accessories > Packs ...,http://www.amazon.co.uk/ClubKing-Ltd-Football-...,"1 Pack of 25 Football Cards, 30 rockstars on e...",Technical Details Item Weight100 g Product Dim...,"1 Pack of 25 Football Cards, 30 rockstars on e...",http://www.amazon.co.uk/ClubKing-Ltd-Football-...,how many cards in a pack it dosn't say..?? // ...,Rock and roll cards // 5.0 // 28 July 2013 // ...,"{""seller""=>[{""Seller_name_1""=>""ClubKing Ltd"", ...",5.0
4,f505611d18b0ae98258896ae1b0f14c8,Crayola 16 Pipsqueak Glitter Glues,Crayola,£5.99,9 new,10,1.0,Characters & Brands > Crayola,http://www.amazon.co.uk/Crayola-3-3542-9-Glitt...,Product Description 16 Pipsqueak washable glit...,Technical Details Item Weight186 g Product Dim...,Product Description 16 Pipsqueak washable glit...,http://www.amazon.co.uk/Crayola-3-3542-9-Glitt...,,Little fingers big imagination // 5.0 // 4 Mar...,"{""seller""=>[{""Seller_name_1""=>""BIC WAREHOUSE"",...",4.2


In [44]:
df_pred = pd.read_csv('Downloads/Test_Data.csv', keep_default_na=False)
df_pred.head()
df_pred_review=pd.read_csv('Downloads/Sample_Submission.csv',keep_default_na=False)

In [45]:
df['average_review_rating'].value_counts()

5.0    3870
4.0     999
4.5     538
4.8     425
4.7     389
4.3     299
4.6     282
4.4     234
4.9     173
4.2     159
4.1     106
3.9       3
3.3       2
3.6       2
3.5       2
3.7       2
3.0       1
Name: average_review_rating, dtype: int64

In [46]:
df['Review'] = (df['customer_reviews'].map(str)).apply(lambda row: row.strip())
#df['Rating'] = [5 if rating > 4.5 else 4 if rating >3.5 else 3 for rating in df['average_review_rating']]
df['Rating'] = df['average_review_rating']
df = df[['Review', 'Rating']]
df.head()

,Review,Rating
0,Item was broken // 3.0 // 2 April 2014 // By\n...,4.5
1,A wonderful collectors piece // 5.0 // 31 Aug....,4.6
2,Great sticker collection // 5.0 // 19 Jun. 201...,4.7
3,Rock and roll cards // 5.0 // 28 July 2013 // ...,5.0
4,Little fingers big imagination // 5.0 // 4 Mar...,4.2


In [47]:
df_pred['Review'] = (df_pred['customer_reviews'].map(str)).apply(lambda row: row.strip())
df_pred = df_pred[['Review']]
df_pred.head()

,Review
0,Dolls house miniatures // 5.0 // 9 Jan. 2013 /...
1,Tangled // 4.0 // 16 May 2011 // By\n \n ...
2,Okay but that's all // 1.0 // 11 July 2011 // ...
3,Good quality but take down carefully as the fo...
4,hilarious // 5.0 // 11 Dec. 2013 // By\n \n...


In [48]:
df = df[df['Review'] != '']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7483 entries, 0 to 7485
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Review  7483 non-null   object 
 1   Rating  7483 non-null   float64
dtypes: float64(1), object(1)
memory usage: 175.4+ KB


In [49]:
df_pred = df_pred[df_pred['Review'] != '']
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2496 entries, 0 to 2495
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  2496 non-null   object
dtypes: object(1)
memory usage: 39.0+ KB


In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['Review']], df['Rating'], random_state=42)
X_train.shape, X_test.shape

((5612, 1), (1871, 1))

In [51]:
X_pred=df_pred[['Review']]
X_pred.shape

(2496, 1)

In [52]:
from collections import Counter
Counter(y_train), Counter(y_test)

(Counter({5.0: 2902,
          4.0: 751,
          4.7: 283,
          4.1: 85,
          4.5: 397,
          4.6: 216,
          4.9: 131,
          4.4: 173,
          4.8: 320,
          4.3: 225,
          4.2: 120,
          3.7: 2,
          3.3: 1,
          3.9: 2,
          3.6: 2,
          3.5: 2}),
 Counter({4.9: 42,
          4.0: 247,
          4.5: 141,
          4.2: 39,
          4.7: 106,
          5.0: 966,
          4.3: 74,
          4.4: 61,
          4.1: 21,
          4.6: 66,
          4.8: 105,
          3.3: 1,
          3.9: 1,
          3.0: 1}))

In [53]:
import string
import textblob

X_train['char_count'] = X_train['Review'].apply(len)
X_train['word_count'] = X_train['Review'].apply(lambda x: len(x.split()))
X_train['word_density'] = X_train['char_count'] / (X_train['word_count']+1)
X_train['punctuation_count'] = X_train['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_train['title_word_count'] = X_train['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_train['upper_case_word_count'] = X_train['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
x_train_snt_obj = X_train['Review'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_train['Polarity'] = [obj.polarity for obj in x_train_snt_obj.values]
X_train['Subjectivity'] = [obj.subjectivity for obj in x_train_snt_obj.values]

X_test['char_count'] = X_test['Review'].apply(len)
X_test['word_count'] = X_test['Review'].apply(lambda x: len(x.split()))
X_test['word_density'] = X_test['char_count'] / (X_test['word_count']+1)
X_test['punctuation_count'] = X_test['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_test['title_word_count'] = X_test['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_test['upper_case_word_count'] = X_test['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
x_test_snt_obj = X_test['Review'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_test['Polarity'] = [obj.polarity for obj in x_test_snt_obj.values]
X_test['Subjectivity'] = [obj.subjectivity for obj in x_test_snt_obj.values]

X_pred['char_count'] = X_pred['Review'].apply(len)
X_pred['word_count'] = X_pred['Review'].apply(lambda x: len(x.split()))
X_pred['word_density'] = X_pred['char_count'] / (X_pred['word_count']+1)
X_pred['punctuation_count'] = X_pred['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_pred['title_word_count'] = X_pred['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_pred['upper_case_word_count'] = X_pred['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
x_pred_snt_obj = X_pred['Review'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_pred['Polarity'] = [obj.polarity for obj in x_pred_snt_obj.values]
X_pred['Subjectivity'] = [obj.subjectivity for obj in x_pred_snt_obj.values]

In [54]:
X_train.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
4129,LFA model // 5.0 // 2 Nov. 2013 // 1 of 1 peop...,524,104,4.990476,36,17,1,0.478125,0.525000
6255,love those rings! // 4.0 // 3 Jun. 2013 // By\...,269,53,4.981481,15,7,2,0.309375,0.525000
641,Four Stars // 4.0 // 30 Nov. 2015 // By\n \...,102,19,5.100000,11,7,0,1.000000,1.000000
5002,"Solid workmanship, easy to assemble, hefty woo...",1124,207,5.403846,72,35,3,0.303565,0.623580
4101,Bicycle Playing Cards // 5.0 // 16 July 2012 /...,1007,191,5.244792,52,40,3,0.376000,0.690556


In [55]:
#Train a linear regression model
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import mean_squared_error
from math import sqrt

classifiers = [
    svm.SVR(),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]
for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train.drop(['Review'], axis=1), y_train,)
    test_predictions = clf.predict(X_test.drop(['Review'], axis=1))
print(test_predictions,'\n')
print("RMSE of linear regression with sentiment analysis feature is", sqrt(mean_squared_error(y_test, test_predictions)))

SVR()
SGDRegressor()
BayesianRidge()
LassoLars()
ARDRegression()
PassiveAggressiveRegressor()
TheilSenRegressor(max_subpopulation=10000)
LinearRegression()
[4.58239818 4.53754369 4.74448843 ... 4.91436557 4.96871495 4.89167496] 

RMSE of linear regression with sentiment analysis feature is 0.34618777525401057


In [56]:
#Now predict with Test_data.csv (which is d_pred and X_pred in code)
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import mean_squared_error
from math import sqrt

classifiers = [
    svm.SVR(),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]
for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train.drop(['Review'], axis=1), y_train,)
    predictions=clf.predict(X_pred.drop(['Review'], axis=1))
print(predictions,'\n')
res = pd.DataFrame(predictions) 
#res.index = pred.index 
res.columns = ["prediction"]
res.to_csv("prediction_results.csv", index = False)     

SVR()
SGDRegressor()
BayesianRidge()
LassoLars()
ARDRegression()
PassiveAggressiveRegressor()
TheilSenRegressor(max_subpopulation=10000)
LinearRegression()
[4.8916873  4.41834462 4.54115116 ... 4.99919315 4.92045589 4.69522198] 



In [57]:
import statistics
print(statistics.mean(y_train))

4.707056307911618


In [58]:
print(statistics.median(y_train[y_train>4.7]))
print(statistics.median(y_train[y_train<=4.7]))

5.0
4.3


In [59]:
y_train_origin=y_train
y_test_origin=y_test
#df['Review'] = (df['Review'].map(str)).apply(lambda row: row.strip())
df['Rating'] = [1 if rating > 4.7 else 0 for rating in df['Rating']]
df = df[['Review', 'Rating']]
df.head()

,Review,Rating
0,Item was broken // 3.0 // 2 April 2014 // By\n...,0
1,A wonderful collectors piece // 5.0 // 31 Aug....,0
2,Great sticker collection // 5.0 // 19 Jun. 201...,0
3,Rock and roll cards // 5.0 // 28 July 2013 // ...,1
4,Little fingers big imagination // 5.0 // 4 Mar...,0


In [60]:
df = df[df['Review'] != '']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7483 entries, 0 to 7485
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  7483 non-null   object
 1   Rating  7483 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 175.4+ KB


In [61]:
df['Rating'].value_counts()

1    4466
0    3017
Name: Rating, dtype: int64

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['Review']], df['Rating'], random_state=42)
X_train.shape, X_test.shape

((5612, 1), (1871, 1))

In [63]:
from collections import Counter
Counter(y_train), Counter(y_test)

(Counter({1: 3353, 0: 2259}), Counter({1: 1113, 0: 758}))

In [64]:
import string

X_train['char_count'] = X_train['Review'].apply(len)
X_train['word_count'] = X_train['Review'].apply(lambda x: len(x.split()))
X_train['word_density'] = X_train['char_count'] / (X_train['word_count']+1)
X_train['punctuation_count'] = X_train['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_train['title_word_count'] = X_train['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_train['upper_case_word_count'] = X_train['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))


X_test['char_count'] = X_test['Review'].apply(len)
X_test['word_count'] = X_test['Review'].apply(lambda x: len(x.split()))
X_test['word_density'] = X_test['char_count'] / (X_test['word_count']+1)
X_test['punctuation_count'] = X_test['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_test['title_word_count'] = X_test['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_test['upper_case_word_count'] = X_test['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [65]:
#Training a logistic regression model
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1, random_state=42, solver='liblinear')
lr.fit(X_train.drop(['Review'], axis=1), y_train)
test_predictions = lr.predict(X_test.drop(['Review'], axis=1))

print(classification_report(y_test, test_predictions))
pd.DataFrame(confusion_matrix(y_test, test_predictions))

              precision    recall  f1-score   support

           0       0.73      0.44      0.55       758
           1       0.70      0.89      0.78      1113

    accuracy                           0.71      1871
   macro avg       0.71      0.66      0.67      1871
weighted avg       0.71      0.71      0.69      1871



,0,1
0,334,424
1,125,988


In [66]:
test_predictions[test_predictions>0]=5
test_predictions[test_predictions==0]=4.3
print(test_predictions)

[4 4 5 ... 5 5 5]


In [67]:
print("RMSE of logistic regression is", sqrt(mean_squared_error(y_test_origin, test_predictions)))

RMSE of logistic regression is 0.49078085449675657


In [68]:
X_train.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count
4129,LFA model // 5.0 // 2 Nov. 2013 // 1 of 1 peop...,524,104,4.990476,36,17,1
6255,love those rings! // 4.0 // 3 Jun. 2013 // By\...,269,53,4.981481,15,7,2
641,Four Stars // 4.0 // 30 Nov. 2015 // By\n \...,102,19,5.100000,11,7,0
5002,"Solid workmanship, easy to assemble, hefty woo...",1124,207,5.403846,72,35,3
4101,Bicycle Playing Cards // 5.0 // 16 July 2012 /...,1007,191,5.244792,52,40,3


In [69]:
import textblob

x_train_snt_obj = X_train['Review'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_train['Polarity'] = [obj.polarity for obj in x_train_snt_obj.values]
X_train['Subjectivity'] = [obj.subjectivity for obj in x_train_snt_obj.values]

x_test_snt_obj = X_test['Review'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_test['Polarity'] = [obj.polarity for obj in x_test_snt_obj.values]
X_test['Subjectivity'] = [obj.subjectivity for obj in x_test_snt_obj.values]

In [70]:
lr.fit(X_train.drop(['Review'], axis=1), y_train, )
test_predictions = lr.predict(X_test.drop(['Review'], axis=1))

print(classification_report(y_test, test_predictions))
pd.DataFrame(confusion_matrix(y_test, test_predictions))

              precision    recall  f1-score   support

           0       0.74      0.52      0.61       758
           1       0.73      0.88      0.79      1113

    accuracy                           0.73      1871
   macro avg       0.73      0.70      0.70      1871
weighted avg       0.73      0.73      0.72      1871



,0,1
0,391,367
1,137,976


In [71]:
test_predictions[test_predictions>0]=5
test_predictions[test_predictions==0]=4.3
print(test_predictions)
print("RMSE of logistic regression with sentiment analysis feature is", sqrt(mean_squared_error(y_test_origin, test_predictions)))      

[4 4 5 ... 5 5 5]
RMSE of logistic regression with sentiment analysis feature is 0.48165712526389276


In [72]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hksca\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hksca\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [73]:
import nltk
import contractions
import re

# remove some stopwords to capture negation in n-grams if possible
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')
stop_words.remove('but')

# load up a simple porter stemmer - nothing fancy
ps = nltk.porter.PorterStemmer()

def simple_text_preprocessor(document): 
    # lower case
    document = str(document).lower()
    
    # expand contractions
    document = contractions.fix(document)
    
    # remove unnecessary characters
    document = re.sub(r'[^a-zA-Z]',r' ', document)
    document = re.sub(r'nbsp', r'', document)
    document = re.sub(' +', ' ', document)
    
    # simple porter stemming
    document = ' '.join([ps.stem(word) for word in document.split()])
    
    # stopwords removal
    document = ' '.join([word for word in document.split() if word not in stop_words])
    
    return document

stp = np.vectorize(simple_text_preprocessor)


In [74]:
X_train['Clean Review'] = stp(X_train['Review'].values)
X_test['Clean Review'] = stp(X_test['Review'].values)

X_train.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,Clean Review
4129,LFA model // 5.0 // 2 Nov. 2013 // 1 of 1 peop...,524,104,4.990476,36,17,1,0.478125,0.525000,lfa model novemb peopl found follow review hel...
6255,love those rings! // 4.0 // 3 Jun. 2013 // By\...,269,53,4.981481,15,7,2,0.309375,0.525000,love ring june eileen june use lot littl ring ...
641,Four Stars // 4.0 // 30 Nov. 2015 // By\n \...,102,19,5.100000,11,7,0,1.000000,1.000000,four star novemb jakki mccarthi novemb excel valu
5002,"Solid workmanship, easy to assemble, hefty woo...",1124,207,5.403846,72,35,3,0.303565,0.623580,solid workmanship easi assembl hefti wood febr...
4101,Bicycle Playing Cards // 5.0 // 16 July 2012 /...,1007,191,5.244792,52,40,3,0.376000,0.690556,bicycl play card juli mr j kimber juli card mu...


In [75]:
X_train_metadata = X_train.drop(['Review', 'Clean Review'], axis=1).reset_index(drop=True)
X_test_metadata = X_test.drop(['Review', 'Clean Review'], axis=1).reset_index(drop=True)

X_train_metadata.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
0,524,104,4.990476,36,17,1,0.478125,0.525000
1,269,53,4.981481,15,7,2,0.309375,0.525000
2,102,19,5.100000,11,7,0,1.000000,1.000000
3,1124,207,5.403846,72,35,3,0.303565,0.623580
4,1007,191,5.244792,52,40,3,0.376000,0.690556


In [76]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1, 1))
X_traincv = cv.fit_transform(X_train['Clean Review']).toarray()
X_traincv = pd.DataFrame(X_traincv, columns=cv.get_feature_names())

X_testcv = cv.transform(X_test['Clean Review']).toarray()
X_testcv = pd.DataFrame(X_testcv, columns=cv.get_feature_names())
X_traincv.head()

,aa,aaa,aaaa,aaaaarghmaz,aaaarrrrrr,aabirch,aah,aall,aardvark,aaron,...,zumbi,zumpano,zunbrown,zupa,zur,zuzana,zxxxx,zygon,zymer,zyrenabud
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
X_train_comb = pd.concat([X_train_metadata, X_traincv], axis=1)
X_test_comb = pd.concat([X_test_metadata, X_testcv], axis=1)

X_train_comb.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,aa,aaa,...,zumbi,zumpano,zunbrown,zupa,zur,zuzana,zxxxx,zygon,zymer,zyrenabud
0,524,104,4.990476,36,17,1,0.478125,0.525000,0,0,...,0,0,0,0,0,0,0,0,0,0
1,269,53,4.981481,15,7,2,0.309375,0.525000,0,0,...,0,0,0,0,0,0,0,0,0,0
2,102,19,5.100000,11,7,0,1.000000,1.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1124,207,5.403846,72,35,3,0.303565,0.623580,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1007,191,5.244792,52,40,3,0.376000,0.690556,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
del(df)
del(X_train)
del(X_test)

In [79]:
lr.fit(X_train_comb, y_train)
test_predictions = lr.predict(X_test_comb)

print(classification_report(y_test, test_predictions))
pd.DataFrame(confusion_matrix(y_test, test_predictions))

              precision    recall  f1-score   support

           0       0.78      0.72      0.75       758
           1       0.82      0.86      0.84      1113

    accuracy                           0.80      1871
   macro avg       0.80      0.79      0.79      1871
weighted avg       0.80      0.80      0.80      1871



,0,1
0,546,212
1,158,955


In [80]:
test_predictions[test_predictions>0]=5
test_predictions[test_predictions==0]=4.3
print(test_predictions)
print("RMSE of logistic regression with bag of words is", sqrt(mean_squared_error(y_test_origin, test_predictions)))      

[5 4 5 ... 5 5 5]
RMSE of logistic regression with bag of words is 0.4352579605821513


In [81]:
#Therefore, linear regression has the best RMSE score. 
#We submit the prediction result from the linear regression model in block 14.